<a href="https://colab.research.google.com/github/oleksandr-kushnir/Coursera_Capstone/blob/master/Assignment_Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto

##1. Cleaning and Preparing the Neighborhoods Data

At first let's import the Pandas library

In [0]:
import pandas as pd

Now we can read the table from the Wikipedia article URL using Pandas native method '.read_html'

In [18]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(wiki_url)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The table contains 'Not assigned' values in the 'Borough' column, let's get rid of this rows:

In [16]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Are there any 'Not assigned' values in the 'Neghborhood' column?

In [5]:
df[df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


Let's check, if there are any NaN values in the DataFrame left:

In [6]:
df[df.isna().any(axis=1)]

,Postal Code,Borough,Neighborhood


The table contains now all the data in the right format, let's look how many rows it has:

In [7]:
print('Number of rows in the dataset:', df.shape[0])

Number of rows in the dataset: 103


##2. Getting the Geospatial Data

We read the geospatial data from the given URL and create a DataFrame:

In [14]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then we join them on our neighborhood DataFrame:

In [17]:
df_merged = df.join(geo_df.set_index('Postal Code'), on='Postal Code')
df_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


##3. Explore and cluster the neighborhoods in Toronto
At first we import all the necessary libraries

In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's create a map of Toronto and our negborhoods on it:

In [58]:
# point in the middle of our neighborhoods
lat_center = df_merged['Latitude'].mean()
long_center = df_merged['Longitude'].mean()

#create map object with center in the middle of neighborhoods
map_toronto = folium.Map(location=[lat_center, long_center], zoom_start=10)

# mark all neighborhoods on the map
for lat, lon, bor, neigh in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
  label = 'Borough: \n{}\n, Neighborhood(s): \n{}\n'.format(bor, neigh)
  folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)


map_toronto

Define Foursquare Credentials and Version: <br>
CLIENT_ID: ************** <br>
CLIENT_SECRRET: ************** <br>
VERSION: 20180605



In [0]:
 # @hidden_cell

CLIENT_ID = 'L1MM1ECQMJI4EWWW0SA2YOKK2D4PI2A1NHENJX2B5J3GAQLX' # your Foursquare ID
CLIENT_SECRET = 'KL1DMABXU2P2FMB5HPGJ3FSD2SE2KXULSRHKPDSCVP4S3HI2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version